Jupyter notebook example to play with functional tests on k8s backend


In [ ]:
import os

os.environ["BUILD_USER_EMAIL"] = "sct.tester@scylladb.com"
os.environ["BUILD_ID"] = "123"
os.environ["SCT_CLUSTER_BACKEND"] = "k8s-local-kind"
os.environ["SCT_CONFIG_FILES"] = "test-cases/scylla-operator/functional.yaml"
os.environ["SCT_SCYLLA_VERSION"] = "2022.2.0"
os.environ["SCT_K8S_USE_CHAOS_MESH"] = "True"
os.environ["SCT_REGION_NAME"] = "eu-west-1"
os.environ["SCT_USE_MGMT"] = "True"
os.environ["SCT_N_DB_NODES"] = "3"
os.environ["SCT_N_LOADERS"] = "1"
os.environ["SCT_N_MONITORS_NODES"] = "0"
assert os.environ["BUILD_USER_EMAIL"] != "sct.tester@scylladb.com", (
    "please use your own email so resources are tracked properly"
)

# logging configuration, for jupyter only (sct logs are intact)
import logging

LOGGER = logging.getLogger(__name__)
LOGGER.setLevel(logging.DEBUG)
consoleHandler = logging.StreamHandler()
consoleHandler.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
consoleHandler.setFormatter(formatter)
LOGGER.addHandler(consoleHandler)

In [ ]:
import os

from functional_tests.scylla_operator.libs.auxiliary import ScyllaOperatorFunctionalClusterTester
from sdcm import sct_abs_path

os.chdir(sct_abs_path(relative_filename=""))
tester_inst = ScyllaOperatorFunctionalClusterTester()
tester_inst.setUpClass()
tester_inst.setUp()

Now to recover to original state and get cluster,can iterate over `db_cluster` fixture, which will yield a `ScyllaPodCluster` object:

In [ ]:
from functional_tests.scylla_operator.conftest import _bring_cluster_back_to_original_state


def db_cluster(tester: ScyllaOperatorFunctionalClusterTester):
    if not tester.healthy_flag:
        raise RuntimeError("cluster is not healthy, skipping rest of the tests")

    with tester.db_cluster.scylla_config_map() as scylla_config_map:
        original_scylla_config_map = scylla_config_map
    original_scylla_cluster_spec = tester.db_cluster.get_scylla_cluster_plain_value("/spec")

    if dataset_name := tester.db_cluster.params.get("k8s_functional_test_dataset"):
        tester.db_cluster.wait_for_nodes_up_and_normal(
            nodes=tester.db_cluster.nodes, verification_node=tester.db_cluster.nodes[0]
        )
        tester.db_cluster.wait_for_init(node_list=tester.db_cluster.nodes, wait_for_db_logs=True)
        tester.db_cluster.prefill_cluster(dataset_name)
    yield tester.db_cluster

    if tester.healthy_flag:
        _bring_cluster_back_to_original_state(
            tester, config_map=original_scylla_config_map, original_scylla_cluster_spec=original_scylla_cluster_spec
        )


cluster = db_cluster(tester_inst).__next__()

Play with it like in provision-test, or run some functional test:


In [ ]:
from functional_tests.scylla_operator.test_functional import test_listen_address

test_listen_address(cluster)